In [1]:
# !pip install ctransformers[cuda]
# !pip3 install numpy==1.26.4
import numpy as np
np.__version__

# restart after

'1.26.3'

In [2]:
import os
# from dotenv import load_dotenv
# load_dotenv('.env')
# hf_api = os.getenv('HF_API')

In [16]:
!pip install transformers==4.38.0
!pip install -q torch accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl
!pip install unstructured ragatouille
# # reranker
from ragatouille import RAGPretrainedModel
!nvcc --version

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 9.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 36.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 1.4 MB/s eta 0:00:00:00:01
  Using cached certifi-2024.2.2-py3-none-any.whl.metadata (2.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 14.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-non

In [17]:
import torch
torch.cuda.is_available()

True

In [5]:
# this will take time :c
!CUDACXX=/usr/local/cuda-12.1/bin/nvcc CMAKE_ARGS="-DLLAMA_CUBLAS=on -DCMAKE_CUDA_ARCHITECTURES=all-major" FORCE_CMAKE=1 pip install llama-cpp-python --no-cache-dir --force-reinstall --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.9/36.9 MB 199.9 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 319.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 281.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 366.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 198.3 MB/s eta 0:00:00a 0:00:01
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.56-cp310-cp310-manylinux_2_26_x86_64.whl size=22629768 sha256=96959f4538094622cad8cdb23481bd0810132457e58ebc5c6b226bb13a0d209d
  Stored in directory: /tmp/pip-ephem-wheel-cache-qp1bt_xg/wheels/e5/09/9d/c413053f6258cb2546cc792418c595e276f9efd5db31a80377
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
  

In [6]:
# fix colab error: https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [9]:
!pip install langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.docstore.document import Document as LangchainDocument
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate, ChatPromptTemplate
import os
import pandas as pd

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 12.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 16.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
!pip install huggingface_hub
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.4/346.4 kB 1.6 MB/s eta 0:00:00:00:01


ModuleNotFoundError: No module named 'transformers'

In [12]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
pd.set_option(
    "display.max_colwidth", None
)

Matplotlib is building the font cache; this may take a moment.


In [ ]:
# pip list --outdated

# Specify the model/versions, etc.
## also specify the knowledge base file and the reference answer directory

In [18]:
EMBEDDING_MODEL = "thenlper/gte-base"
RERANKER_MODEL = "colbert-ir/colbertv2.0"
RERANKER = RAGPretrainedModel.from_pretrained(RERANKER_MODEL)

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [19]:
EMBEDDING_MODEL_NAME = EMBEDDING_MODEL
embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},
)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/219M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
!pwd

/home/ec2-user/SageMaker/src-rag-final


In [21]:
FAISS_FILE = '../faiss_index_total_final'
# num_retrieved_docs: int = 10
# num_docs_final: int = 3
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(FAISS_FILE, embedding_model, allow_dangerous_deserialization = True)

In [22]:
model_path = "TheBloke/Llama-2-7B-Chat-GGUF"
model_basename = "llama-2-7b-chat.Q4_K_M.gguf"
model_path = hf_hub_download(repo_id=model_path, filename=model_basename)

llama-2-7b-chat.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

# Reader LLM

In [39]:
# https://www.reddit.com/r/LocalLLaMA/comments/1343bgz/what_model_parameters_is_everyone_using/
# temperature	0.7
# repetition_penalty=	1.176
# top_k	= 40
# top_p= 0.1

# Initialize Large Language Model for answer generation
llm_answer_gen = LlamaCpp(
streaming = True,
model_path = model_path,
temperature=.2,
n_gpu_layers=-1,
top_p=.1,
top_k	= 40,
repetition_penalty=	1.176,
verbose=True,
n_ctx=2400
)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! repetition_penalty is not default parameter.
                repetition_penalty was transferred to model_kwargs.
                Please confirm that repetition_penalty is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ec2-user/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_lo

In [40]:
llm_answer_gen.verbose = False

# RAG Function

In [25]:
# link: https://python.langchain.com/docs/use_cases/chatbots/retrieval

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

def answer_llama_new(
    question: str,
    knowledge_index: FAISS,
    reranker: RAGPretrainedModel,
    num_retrieved_docs: int = 10,
    num_docs_final: int = 3,
    llm: LlamaCpp = llm_answer_gen,
    ): #-> Tuple[str, List[LangchainDocument]]

    # https://stackoverflow.com/questions/76551067/how-to-create-a-langchain-doc-from-an-str

    print("=> Retrieving documents...")
    relevant_docs_acquired = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_doc_id = [doc.metadata['source'] for doc in relevant_docs_acquired]

    if reranker:
        print("=> Reranking documents...")
        relevant_docs = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)

        relevant_docs = [LangchainDocument(page_content=doc["content"],
                                           metadata={'source': relevant_doc_id[ doc['result_index']] } ) for doc in relevant_docs]

        relevant_docs_content = [doc.page_content for doc in relevant_docs]
        relevant_doc_id = [doc.metadata for doc in relevant_docs]

    else:
        relevant_docs_content = [doc.page_content for doc in relevant_docs_acquired]
        relevant_docs = relevant_docs_acquired

    relevant_docs_content = relevant_docs_content[:num_docs_final]
    relevant_docs_indexed = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs_content)])

    context_and_question = f"Please provide concise answers while extracting only the necessary keywords from the context need to answer the question. Keep responses as short as possible and avoid verbosity. Include multiple correct answers if necessary. If no relevant context, then say 'I dont know'. \n\nContext:\n{context}\n\nQuestion:\n{question}\n\nAnswer:"
    
    answer = llm.invoke(context_and_question)
    
    if answer is not None:
        new_answer = answer.split(': ', 1)
        if len(new_answer) > 1:
            answer = new_answer[1]
        else:
            answer = new_answer[0]
    # print(f"Question: {question}")
    
    forbidden_list = ["According to the provided context,", "Based on the provided context,", '\n']
    for phrase in forbidden_list:
        answer = answer.replace(phrase, " ")
    answer = answer.replace("<pad>", "").replace("</s>", "").replace("\n", "").strip()
    # print(f"Answer: {answer}")
    return answer, relevant_docs_content

## Evaluation

In [26]:
# specify output csv 
input_file = 'qa_gold-llama-no_chat_template.csv'
output_file = 'llama-no_chat_template-output.csv'
csv_input_dir = f'csv_qa_gold/{input_file}'
csv_output_dir = f'csv_qa_gold/{output_file}'

In [27]:
def initialize_model_answer_column(df):
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = None
    return df

In [28]:
import glob
import os
import pandas as pd

directory_csv = 'csv_qa_gold'
csv_files = ['csv_qa_gold/test_combined.csv']

# read in the csv files in the directory and concatenate
df_total = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    print(df.shape)
    df_total = pd.concat([df_total, df], axis=0)
print(df_total.shape)
df_total = initialize_model_answer_column(df_total)
df_total.to_csv(csv_input_dir, index=False) 
# doing this so the original is untouched

(191, 3)
(191, 3)


In [29]:
df_total.iloc[10:20, :]  # please check that excel does not fuck up for row 15,16 the November 2006 to Nov-06 or some other format

,Category,Question,Answer,ModelAnswer
10,webpages,Was the Scottish terrier a familiar figure at CMU before it officially became a mascot?,Yes,None
11,webpages,Is there an official mascot costume of Scotty?,Yes,None
12,webpages,Who led the effort to make Carnegie Mellon's mascot the Scottish Terrier?,Susan Bassett and Jennifer Church,None
13,webpages,Where did Scotty debug as CMU's official mascot?,at a football game,None
14,webpages,Who did CMU partner with to create the Scotty mascot graphics?,SME Branding,None
15,webpages,What month and year was the Mascot Identity Task Force formed?,November 2006,None
16,webpages,When did Scotty debut as CMU's official mascot?,November 2007,None
17,webpages,What characteristics does the Scottish terrier represent?,"determined, thoughtful, strength, power, agility in a small package",None
18,webpages,Where did the athletic director of CMU graduate from?,Brandeis,None
19,webpages,When was Carnegie Technical Schools founded?,1900,None


In [30]:
def generate_answer(question,llm, return_relevant_docs = False):
    answer, relevant_docs = answer_llama_new(question, KNOWLEDGE_VECTOR_DATABASE,
                            reranker=RERANKER, llm = llm)
    if return_relevant_docs:
        return answer, relevant_docs
    return answer

In [31]:
# Function to run over a list of questions and return the answers
# Define the function to process the CSV and add model answers

def generate_answers_and_save(csv_path, llm):
    df = pd.read_csv(csv_path)
    if 'ModelAnswer' not in df.columns:
        df['ModelAnswer'] = ''
    for index, row in df.iterrows():
        ModelAnswer = generate_answer(row['Question'],llm=llm)
        df.at[index, 'ModelAnswer'] = ModelAnswer
        df.to_csv(csv_output_dir, index=False)
    return df

In [32]:
df_with_model_answers = generate_answers_and_save(csv_path=csv_input_dir, llm = llm_answer_gen)
print(df_with_model_answers.shape)
df_with_model_answers.head()

=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 15.65it/s]

llama_print_timings:        load time =     652.09 ms
llama_print_timings:      sample time =      14.89 ms /    23 runs   (    0.65 ms per token,  1544.14 tokens per second)
llama_print_timings: prompt eval time =   14109.95 ms /  1463 tokens (    9.64 ms per token,   103.69 tokens per second)
llama_print_timings:        eval time =     568.80 ms /    22 runs   (   25.85 ms per token,    38.68 tokens per second)
llama_print_timings:       total time =   15186.17 ms /  1485 tokens


=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


100%|██████████| 1/1 [00:00<00:00, 16.08it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =     652.09 ms
llama_print_timings:      sample time =      12.98 ms /    20 runs   (    0.65 ms per token,  1540.48 tokens per second)
llama_print_timings: prompt eval time =     261.24 ms /    26 tokens (   10.05 ms per token,    99.52 tokens per second)
llama_print_timings:        eval time =     493.13 ms /    19 runs   (   25.95 ms per token,    38.53 tokens per second)
llama_print_timings:       total time =     833.88 ms /    45 tokens


=> Retrieving documents...
=> Reranking documents...
Your documents are roughly 213.6 tokens long at the 90th percentile! This is quite long and might slow down reranking!
 Provide fewer documents, build smaller chunks or run on GPU if it takes too long for your needs!


...

100%|██████████| 1/1 [00:00<00:00, 81.43it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =     652.09 ms
llama_print_timings:      sample time =      34.29 ms /    52 runs   (    0.66 ms per token,  1516.52 tokens per second)
llama_print_timings: prompt eval time =    1739.30 ms /   196 tokens (    8.87 ms per token,   112.69 tokens per second)
llama_print_timings:        eval time =    1150.93 ms /    51 runs   (   22.57 ms per token,    44.31 tokens per second)
llama_print_timings:       total time =    3150.32 ms /   247 tokens


=> Retrieving documents...
=> Reranking documents...


100%|██████████| 1/1 [00:00<00:00, 70.07it/s]
Llama.generate: prefix-match hit

llama_print_timings:        load time =     652.09 ms
llama_print_timings:      sample time =      38.42 ms /    58 runs   (    0.66 ms per token,  1509.79 tokens per second)
llama_print_timings: prompt eval time =    1819.17 ms /   206 tokens (    8.83 ms per token,   113.24 tokens per second)
llama_print_timings:        eval time =    1285.26 ms /    57 runs   (   22.55 ms per token,    44.35 tokens per second)
llama_print_timings:       total time =    3393.44 ms /   263 tokens


(191, 4)


,Category,Question,Answer,ModelAnswer
0,webpages,Do students in Kiltie have to memorize music?,No,"No, the music is changed for every show. You should invest in a good and trusty lyre."
1,webpages,Do I need to email my home address to the Kiltie Band Director if I'm interested in joining?,Yes,"No, you do not need to email your home address to the Kiltie Band Director."
2,webpages,Do I need to audition to join the Kiltie Band?,No,"No, any member of the campus community with music experience is able to join the Kiltie Band!"
3,webpages,Where does the Kiltie Band rehearse?,CUC Studio Theater,The Kiltie Band rehearses at the CUC Studio Theater.
4,webpages,When should I get an instrument from the Kiltie Band if I need one?,4:30p.m. before the first rehearsal,You should invest in a good and trusty lyre.


In [35]:
# with additional extraction prompt after generation without prompt
type(df_with_model_answers['ModelAnswer'][0]),type(df_with_model_answers['Answer'][0])
df_with_model_answers = df_with_model_answers.astype(str)  # Convert columns to string type
from evaluation_csv import total_score_csv
exact_match, f1_score, recall_score = total_score_csv(df_with_model_answers['ModelAnswer'], df_with_model_answers['Answer'])
print(f'Exact match: {exact_match}')
print(f'F1 score: {f1_score}')
print(f'Recall score: {recall_score}')

Exact match: 0.04712041884816754
F1 score: 0.19903977874921847
Recall score: 0.45991834195224224


In [36]:
for category in df['Category'].unique():
    print(f'Category: {category}')
    df_category = df_with_model_answers[df_with_model_answers['Category'] == category]
    exact_match, f1_score, recall_score = total_score_csv(df_category['ModelAnswer'], df_category['Answer'])
    print(f'Exact match: {exact_match}')
    print(f'F1 score: {f1_score}')
    print(f'Recall score: {recall_score}')
    print('\n')

Category: webpages
Exact match: 0.09259259259259259
F1 score: 0.31618458605389715
Recall score: 0.6935956790123456


Category: tabular_webpages
Exact match: 0.0
F1 score: 0.14436440295815298
Recall score: 0.46610576923076924


Category: other_pdf
Exact match: 0.0
F1 score: 0.13257874211960932
Recall score: 0.2637853692201518


Category: papers_pdf
Exact match: 0.0
F1 score: 0.0574977817213842
Recall score: 0.22222222222222224


Category: schedule_pdf
Exact match: 0.0
F1 score: 0.015810276679841896
Recall score: 0.045454545454545456


Category: jsons
Exact match: 0.05714285714285714
F1 score: 0.2100834755820694
Recall score: 0.49030412753234165


Category: json_hard
Exact match: 0.0
F1 score: 0.030124777183600716
Recall score: 0.048863636363636366




In [38]:
df_with_model_answers.to_csv('../llama-no-temp-output.csv', index=False)

In [33]:
display(df_with_model_answers.head(60))

,Category,Question,Answer,ModelAnswer
0,webpages,Do students in Kiltie have to memorize music?,No,"No, the music is changed for every show. You should invest in a good and trusty lyre."
1,webpages,Do I need to email my home address to the Kiltie Band Director if I'm interested in joining?,Yes,"No, you do not need to email your home address to the Kiltie Band Director."
2,webpages,Do I need to audition to join the Kiltie Band?,No,"No, any member of the campus community with music experience is able to join the Kiltie Band!"
3,webpages,Where does the Kiltie Band rehearse?,CUC Studio Theater,The Kiltie Band rehearses at the CUC Studio Theater.
4,webpages,When should I get an instrument from the Kiltie Band if I need one?,4:30p.m. before the first rehearsal,You should invest in a good and trusty lyre.
5,webpages,When is Andrew Carnegie's birthday?,"November 25th, 1835","Andrew Carnegie was born in Dunfermline, Scotland in 1835. His birthday is on November 25th."
6,webpages,What time are rehearsals on Game Days for the Kiltie Band?,9:30-11:00 a.m.,Rehearsals for the Kiltie Band on Game Days are at 9:30-11 a.m.
7,webpages,How should Kilties promote the success of the Kiltie Band?,"attendance, attitude, and dependabiliy","Kilties should promote the success of the Kiltie Band by attending rehearsals consistently, having a positive attitude, and being dependable."
8,webpages,What should I email to the band director if I'm interested in joining the Kiltie Band?,"name, high school, address at Carnegie Mellon, home address, cell number, home number, whether you plan on being in the band or colorguard, if in the band what instrument(s) you play, and whether you need to borrow an instrument or equipment","If you are interested in joining the Kiltie Band, please email the following information to the band director, Jeremy Olisar: Name High School Address at Carnegie Mellon (if known) Home address Cell number Home number Whether you plan on being in the band or colorguard If in the band what instrument(s) you play Whether you need to borrow an instrument or equipment To see and hear the band in action, visit our YouTube channel ."
9,webpages,What university is the Kiltie Band a part of?,Carnegie Mellon,The Kiltie Band is a part of Carnegie Mellon University.
